# Problem 9.3

In [1]:
clear
set obs 8
egen block = repeat(), values("A")
replace block ="B" in 5/8




number of observations (_N) was 0, now 8


(4 real changes made)


In [3]:
	input int y0 int y1
	0 2
	1 5
	1 3
	2 1	
	2 3
	3 3
	4 9
	4 7 end

stata_kernel error: code entered was incomplete.

This usually means that a loop or program was not correctly terminated.
This can also happen if you are in `#delimit ;` mode and did not end the
command with `;`. Use `%delimit` to see the current delimiter mode and
use `#delimit cr` to switch back to the default mode where `;` is
unnecessary.


In [6]:
// function to calculate population covariance
cap program drop cov_pop
program define cov_pop, rclass
args x y	
tempvar xy_dev 
qui sum `x'
local avg_x = r(mean)
local length = r(N)	

qui sum `y'
local avg_y = r(mean)
	
gen `xy_dev' = (`x'-`avg_x')*(`y'-`avg_y')
qui tabstat `xy_dev', stat(sum) save
return scalar cor_pop = el(r(StatTotal),1,1)/`length'
end

qui egen rank_y1=rank(y1), unique
qui gen id=_n

In [7]:
vlookup id, generate(y1_lowtohigh) key(rank_y1) value(y1)
replace id = 9-id
vlookup id, generate(y1_hightolow) key(rank_y1) value(y1)



(8 real changes made)



In [8]:
cov_pop y0 y1_hightolow
di "cov.min ="%8.3f r(cor_pop)



cov.min =  -3.141


In [9]:
cov_pop y0 y1_lowtohigh
di "cov.min ="%8.3f r(cor_pop)



cov.min =   3.234


In [10]:
qui replace id=_n
qui replace id=. if block=="B"
qui egen rank_y1_A = rank(y1), unique by(block)
qui replace rank_y1_A =. if block=="B"
qui vlookup id, generate(y1_hightolow_block_A) key(rank_y1_A) value(y1)
qui replace id = _n-4
qui replace id =. if block=="A"
qui egen rank_y1_B = rank(y1), unique by(block)
qui replace rank_y1_B =. if block=="A"
qui vlookup id, generate(y1_hightolow_block_B) key(rank_y1_B) value(y1)
qui gen y1_hightolow_block = y1_hightolow_block_A
qui replace y1_hightolow_block = y1_hightolow_block_B if block=="B"
qui replace id=5-_n
qui replace id=. if block=="B"
qui vlookup id, generate(y1_lowtohigh_block_A) key(rank_y1_A) value(y1)
qui replace id = 9-_n
qui replace id =. if block=="A"
qui vlookup id, generate(y1_lowtohigh_block_B) key(rank_y1_B) value(y1)
qui gen y1_lowtohigh_block = y1_lowtohigh_block_A
qui replace y1_lowtohigh_block = y1_lowtohigh_block_B if block=="B"

In [11]:
cov_pop y0 y1_lowtohigh_block
di "cov.min ="%8.4f r(cor_pop)



cov.min = -0.0156


In [12]:
cov_pop y0 y1_hightolow_block
di "cov.min ="%8.3f r(cor_pop)



cov.min =   2.984
